# Backtesting Flowchart
- Add charts like x = 1, y = 7 We can analyze the indicators easier. 
- Step one getting the high quality processed data
- Step two Strategy selection part 
- Make the calculations give the profit rate & win-lose values
- Change the data interval to hourly, 30 min, 10 min & check the results again
- Create a loop for large scale backtesting
- Create a model for developing new indicators

In [4]:
import yfinance as yf, pandas as pd, matplotlib.pyplot as plt, pandas_ta as ta, numpy as np
import time
import os, math, time, warnings
from datetime import datetime, timedelta
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Don't wrap columns
pd.set_option('display.max_colwidth', None) # Show full content of each column
pd.set_option('display.float_format', lambda x: '%.2f' % x)

def single_stock_analysis(symbol, period="2y"):
    """
    Analyze a single stock with all indicators over specified period
    
    Parameters:
    symbol: Stock symbol (e.g., "AAPL", "MGROS.IS")
    period: Time period ("1y", "2y", "5y", "max")
    """
    
    try:
        print(f"Downloading data for {symbol}...")
        ticker = yf.Ticker(symbol)
        
        # Get stock info
        stock_info = ticker.info
        basic_info = {
            "symbol": stock_info.get("symbol", symbol),
            "priceToBook": stock_info.get("priceToBook", None),
            "bookValue": stock_info.get("bookValue", None),
            "recommendationKey": stock_info.get("recommendationKey", None),
            "fiftyTwoWeekLow": stock_info.get("fiftyTwoWeekLow", None),
            "fiftyTwoWeekHigh": stock_info.get("fiftyTwoWeekHigh", None),
            "targetHighPrice": stock_info.get("targetHighPrice", None),
            "targetLowPrice": stock_info.get("targetLowPrice", None),
            "targetMeanPrice": stock_info.get("targetMeanPrice", None)
        }
        
        # Get historical data
        historical_data = ticker.history(period=period)
        
        if historical_data.empty:
            print(f"No historical data found for {symbol}")
            return None
        
        print(f"Processing {len(historical_data)} days of data...")
        
        # Calculate all indicators
        print("Calculating RSI...")
        historical_data['RSI'] = ta.rsi(historical_data['Close'], length=14)
        
        print("Calculating ADX...")
        adx_data = ta.adx(historical_data['High'], historical_data['Low'], historical_data['Close'], length=14)
        historical_data['ADX'] = adx_data['ADX_14']
        
        print("Calculating MACD...")
        macd = ta.macd(historical_data['Close'], fast=12, slow=26, signal=9)
        historical_data = pd.concat([historical_data, macd], axis=1)
        historical_data['MACDAS'] = historical_data['MACD_12_26_9'] - historical_data['MACDs_12_26_9']
        historical_data['MACDAS_Signal'] = historical_data['MACDAS'].ewm(span=9, adjust=False).mean()

        historical_data['CCI'] = ta.cci(historical_data['High'], historical_data['Low'], historical_data['Close'], length=20)
        historical_data['ROC'] = ta.roc(historical_data['Close'], length=12)
        historical_data['ATR'] = ta.atr(historical_data['High'], historical_data['Low'], historical_data['Close'], length=14)
        historical_data['MFI'] = ta.mfi(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'], length=14)
        historical_data["FWMA"] = ta.fwma(historical_data['Close'], length=14)
        historical_data['OBV'] = ta.obv(historical_data['Close'], historical_data['Volume'])
        historical_data['CMF'] = ta.cmf(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'])
        historical_data['AD'] = ta.ad(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'])
        historical_data["VWAP"] = ta.vwap(historical_data['High'], historical_data['Low'], historical_data['Close'], historical_data['Volume'])
        historical_data["kama"] = ta.kama(historical_data['Close'], length=14)
        supertrend = ta.supertrend(historical_data['High'], historical_data['Low'], historical_data['Close'], length=10, multiplier=3)
        historical_data['Supertrend'] = supertrend['SUPERT_10_3.0']
        historical_data['Supertrend_direction'] = supertrend['SUPERTd_10_3.0']
        historical_data['EMA_200'] = ta.ema(historical_data['Close'], length=200)
        ichimoku = ta.ichimoku(historical_data['High'], historical_data['Low'], historical_data['Close'], 
                              tenkan=9, kijun=26, senkou=52, offset=26, include_chikou=True)
        
        if isinstance(ichimoku, tuple) and len(ichimoku) >= 2:
            ichi_df1, ichi_df2 = ichimoku[0], ichimoku[1]
            historical_data['Tenkan'] = ichi_df1['ITS_9'] if 'ITS_9' in ichi_df1.columns else None
            historical_data['Kijun'] = ichi_df1['IKS_26'] if 'IKS_26' in ichi_df1.columns else None
            historical_data['Senkou_A'] = ichi_df1['ISA_9'] if 'ISA_9' in ichi_df1.columns else None
            historical_data['Senkou_B'] = ichi_df1['ISB_26'] if 'ISB_26' in ichi_df1.columns else None
            historical_data['Chikou'] = ichi_df2['ICS_26'] if 'ICS_26' in ichi_df2.columns else None
        else:
            historical_data['Tenkan'] = None
            historical_data['Kijun'] = None
            historical_data['Senkou_A'] = None
            historical_data['Senkou_B'] = None
            historical_data['Chikou'] = None
        stoch = ta.stoch(historical_data['High'], historical_data['Low'], historical_data['Close'], k=14, d=3)
        historical_data['Stoch_K'] = stoch['STOCHk_14_3_3']
        historical_data['Stoch_D'] = stoch['STOCHd_14_3_3']
        stochrsi = ta.stochrsi(historical_data['Close'], length=14, rsi_length=14, k=3, d=3)
        historical_data['StochRSI_K'] = stochrsi['STOCHRSIk_14_14_3_3']
        historical_data['StochRSI_D'] = stochrsi['STOCHRSId_14_14_3_3']
        bollinger = ta.bbands(historical_data['Close'], length=20, std=2)
        historical_data['BB_Lower'] = bollinger.iloc[:, 0]
        historical_data['BB_Middle'] = bollinger.iloc[:, 1]
        historical_data['BB_Upper'] = bollinger.iloc[:, 2]
        historical_data['BB_BWidth'] = bollinger.iloc[:, 3]
        historical_data['BB_%B'] = bollinger.iloc[:, 4]
        print("Calculating derived metrics...")
        historical_data["YF%"] = ((historical_data["Close"] - historical_data["Close"].min()) / 
                                 (historical_data["Close"].max() - historical_data["Close"].min())) * 100
        historical_data["MACDAS_dif"] = historical_data["MACDAS"] - historical_data["MACDAS_Signal"]
        historical_data["change"] = historical_data["Close"].pct_change() * 100
        historical_data["BB_Pot"] = ((historical_data['BB_Upper'] / historical_data["Close"]) - 1) * 100
        historical_data["BB_Opt"] = ((historical_data['BB_Lower'] / historical_data["Close"]) - 1) * 100
        
        # Trend analysis
        historical_data["TrendWay"] = np.select([
            (historical_data["ADX"] > 20) & (historical_data["ROC"] > 0),
            (historical_data["ADX"] > 20) & (historical_data["ROC"] <= 0),
            historical_data["ADX"] <= 20
        ], ["upper", "lower", "no-trend"], default="unknown")
        
        # Ichimoku Cloud Analysis
        historical_data["Cloud_Position"] = np.select([
            (historical_data["Close"] > historical_data["Senkou_A"]) & 
            (historical_data["Close"] > historical_data["Senkou_B"]) & 
            historical_data["Senkou_A"].notna() & historical_data["Senkou_B"].notna(),
            (historical_data["Close"] < historical_data["Senkou_A"]) & 
            (historical_data["Close"] < historical_data["Senkou_B"]) & 
            historical_data["Senkou_A"].notna() & historical_data["Senkou_B"].notna()
        ], ["Above_Cloud", "Below_Cloud"], default="No_Data")
        
        historical_data["TK_Cross"] = np.select([
            (historical_data["Tenkan"] > historical_data["Kijun"]) & 
            historical_data["Tenkan"].notna() & historical_data["Kijun"].notna(),
            (historical_data["Tenkan"] < historical_data["Kijun"]) & 
            historical_data["Tenkan"].notna() & historical_data["Kijun"].notna()
        ], ["Bullish", "Bearish"], default="No_Data")
        
        # Add basic stock info as columns (for the latest data)
        for key, value in basic_info.items():
            historical_data[key] = value
        
        # Add timestamp
        historical_data['analysis_time'] = datetime.now()
        
        print(f"Analysis complete! Data shape: {historical_data.shape}")
        
        # Create CSVs directory if it doesn't exist and save to CSV
        csv_dir = "Backtesting_CSVs"
        if not os.path.exists(csv_dir):
            os.makedirs(csv_dir)
            print(f"Created directory: {csv_dir}")
        
        now = datetime.now().strftime("%d_%m_%y")
        filename = f"{csv_dir}/{symbol}_{period}_{now}.csv"
        historical_data.to_csv(filename)
        print(f"Data saved to: {filename}")
        
        # Display summary of latest values
        latest_data = historical_data.iloc[-1]
        print(f"\n=== Latest Values for {symbol} ===")
        print(f"Close Price: {latest_data['Close']:.2f}")
        print(f"RSI: {latest_data['RSI']:.2f}")
        print(f"ADX: {latest_data['ADX']:.2f}")
        print(f"MACD: {latest_data['MACD_12_26_9']:.4f}")
        print(f"Cloud Position: {latest_data['Cloud_Position']}")
        print(f"TK Cross: {latest_data['TK_Cross']}")
        print(f"Trend Way: {latest_data['TrendWay']}")
        print(f"Supertrend Direction: {latest_data['Supertrend_direction']}")
        
        return historical_data
        
    except Exception as e:
        print(f"Error analyzing {symbol}: {e}")
        return None

def analyze_multiple_stocks(stock_list, period="max"):
    """
    Analyze multiple stocks one by one
    """
    results = {}
    for i, symbol in enumerate(stock_list, 1):
        print(f"\n{'='*60}")
        print(f"Analyzing {i}/{len(stock_list)}: {symbol}")
        print(f"{'='*60}")
        
        result = single_stock_analysis(symbol, period)
        if result is not None:
            results[symbol] = result
        
        # Small delay to avoid overwhelming the API
        time.sleep(1)
    
    return results


# Example usage:
if __name__ == "__main__":
    # Define your stock lists (from your original code)
    portfolio = ["MGROS.IS","KCHOL.IS","THYAO.IS","KRDMD.IS","CCOLA.IS","BRSAN.IS","ZOREN.IS","TTRAK.IS","AEFES.IS","FROTO.IS","ALCAR.IS"]
    bist100 = ['AEFES.IS','AGHOL.IS','AHGAZ.IS','AKBNK.IS','AKCNS.IS','AKFGY.IS','AKFYE.IS','AKSA.IS','AKSEN.IS','ALARK.IS','ALBRK.IS','ALFAS.IS','ARCLK.IS','ASELS.IS','ASTOR.IS','BERA.IS','BIENY.IS','BIMAS.IS','BRSAN.IS','BRYAT.IS','BUCIM.IS','CANTE.IS','CCOLA.IS','CIMSA.IS','CWENE.IS','DOAS.IS','DOHOL.IS','ECILC.IS','ECZYT.IS','EGEEN.IS','ENJSA.IS','ENKAI.IS','EREGL.IS','EUPWR.IS','EUREN.IS','FROTO.IS','GARAN.IS','GENIL.IS','GESAN.IS','GLYHO.IS','GUBRF.IS','HALKB.IS','HEKTS.IS','IMASM.IS','IPEKE.IS','ISCTR.IS','ISDMR.IS','ISMEN.IS','IZMDC.IS','KARSN.IS','KAYSE.IS','KCAER.IS','KCHOL.IS','KMPUR.IS','KONTR.IS','KONYA.IS','KORDS.IS','KOZAA.IS','KOZAL.IS','KRDMD.IS','KZBGY.IS','MAVI.IS','MGROS.IS','MIATK.IS','ODAS.IS','OTKAR.IS','OYAKC.IS','PENTA.IS','PETKM.IS','PGSUS.IS','QUAGR.IS','SAHOL.IS','SASA.IS','SISE.IS','SKBNK.IS','SMRTG.IS','SOKM.IS','TAVHL.IS','TCELL.IS','THYAO.IS','TKFEN.IS','TOASO.IS','TSKB.IS','TTKOM.IS','TTRAK.IS','TUKAS.IS','TUPRS.IS','ULKER.IS','VAKBN.IS','VESBE.IS','YEOTK.IS','YKBNK.IS','YYLGD.IS','ZOREN.IS']

    # Example 1: Analyze a single stock for 2 years
    #print("=== Single Stock Analysis ===")
    #apple_data = single_stock_analysis("SAHOL.IS", period="5y")
    
    # Example 2: Analyze your portfolio
    print("\n=== Portfolio Analysis ===")
    portfolio_results = analyze_multiple_stocks(bist100[:94], period="max")  # First 3 stocks for demo
    
    # Example 3: Get maximum available data
    #print("\n=== Maximum Historical Data ===")
    #max_data = single_stock_analysis("MGROS.IS", period="max")




=== Portfolio Analysis ===

Analyzing 1/94: AEFES.IS
Processing 6482 days of data...
Calculating RSI...
Calculating ADX...
Calculating MACD...
Calculating derived metrics...
Analysis complete! Data shape: (6482, 59)
Data saved to: Backtesting_CSVs/AEFES.IS_max_22_07_25.csv

=== Latest Values for AEFES.IS ===
Close Price: 15.66
RSI: 57.61
ADX: 18.00
MACD: 0.2554
Cloud Position: No_Data
TK Cross: Bearish
Trend Way: no-trend
Supertrend Direction: 1

Analyzing 2/94: AGHOL.IS
Processing 6481 days of data...
Calculating RSI...
Calculating ADX...
Calculating MACD...
Calculating derived metrics...
Analysis complete! Data shape: (6481, 59)
Data saved to: Backtesting_CSVs/AGHOL.IS_max_22_07_25.csv

=== Latest Values for AGHOL.IS ===
Close Price: 279.75
RSI: 60.75
ADX: 17.22
MACD: 3.3357
Cloud Position: No_Data
TK Cross: Bearish
Trend Way: no-trend
Supertrend Direction: 1

Analyzing 3/94: AHGAZ.IS
Processing 638 days of data...
Calculating RSI...
Calculating ADX...
Calculating MACD...
Calculatin